<a href="https://colab.research.google.com/github/Locomody/Automata_Theory/blob/main/Acceptor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of acceptor model on python.

Acceptor is five-element tuple $\mathcal{A} = (\Sigma,\  \mathcal{S},\  \mathcal{s_0},\  \mathcal{F} \subset \mathcal{S},\  \delta: \mathcal{S} \times \Sigma \rightarrow \mathcal{S})$ where:
* $\sigma$ - finite alphabet 
* $\mathcal{S}$ - finite set of states 
* $\mathcal{s_0}$ - the initial state
* $\mathcal{F} \subset \mathcal{S}$ - finite set of final states
* $\delta: \mathcal{S} \times \Sigma \rightarrow \mathcal{S}$ - transition function

In [3]:
from typing import Set, Dict, Union, List


class Acceptor:

    def __init__(self,
                 alphabet: Set[str],
                 states: Set[str],
                 initial_state: str,
                 accepting_states: Set[str],
                 transition_function: Dict[str, Dict[str, str]]
                 ):

        self._alphabet = alphabet
        self._states = states
        self._initial_state = initial_state
        self._accepting_states = accepting_states
        self._transition_function = transition_function

    def accept(self, word: Union[List[str], str]) -> bool:
        """
        Check whether the acceptor accepts the word(iteratively).
        
        :param word: the list of symbols.
        :return: True if the acceptor accepts the word, False if not.
        """

        curr_state = self._initial_state

        for symbol in word:

            if symbol not in self._alphabet:
                return False

            elif self._transition_function.get(curr_state).get(symbol) is not None:
                curr_state = self._transition_function.get(curr_state).get(symbol)

            else:
                return False
        return curr_state in self._accepting_states
    
    def accept_reccursive(self, word):
        """
        Check whether the acceptor accepts the word(recursively).

        :param word: the list of symbols.
        :return: True if the acceptor accepts the word, False if not.
        """
        def rec(word, curr_state):

            if len(word) == 0:
                return curr_state in self._accepting_states
        
            else:
                curr_state = self._transition_function[curr_state][word[0]]
                return rec(word[1::], curr_state)
        
        return rec(word, self._initial_state)
      

    @property
    def alphabet(self):
        return self._alphabet
    
    @property
    def states(self):
        return self._states
      
    @property
    def initial_state(self):
        return self._initial_state
    
    @property
    def accepting_states(self):
        return self._accepting_states
    
    @property
    def transition_function(self):
        return self._transition_function


# Example of simple acceptor that accepts series of 1s and 0s if it is even number of 1s in series
alphabet = {"1", "0"}
states = {"even", "odd", "s3", "s4"}
initial_state = "even"
accepting_states = {"even"}
transition_function = {
    "even": {
        "0": "even",
        "1": "odd"
    },
    "odd": {
        "0": "odd",
        "1": "even"
    },
}
even_ones = Acceptor(alphabet, states, initial_state, accepting_states, transition_function)
words = ["1101111", "1101011101", "100010101110101"]
tuple(even_ones.accept(word) for word in words)


(True, False, True)

#Delete unreachable states

The state $\mathcal{s}$  of acceptor called unreachable if no string $\mathcal{w} \in \Sigma^*$ for wich $p = \delta^*(\mathcal{s_0}, w)$ 

Algorithm

In [4]:
def reachable(acceptor: Acceptor):
    """
    Delete all unreacheble states from acceptor

    :param acceptor: acceptor in wich we want to delete unreachable states
    :return: new acceptor without unreacheble states
    """
    reachable_states = {acceptor.initial_state}
    states = {acceptor.initial_state}

    while states:
        temp = {acceptor.transition_function[state][symbol] for state in states for symbol in acceptor.alphabet}
        states = temp - reachable_states
        reachable_states = reachable_states | states
    
    return Acceptor(acceptor.alphabet, reachable_states, acceptor.initial_state, acceptor.accepting_states, acceptor.transition_function)

print(even_ones.states)
even_ones = reachable(even_ones)
even_ones.states

{'s3', 'even', 'odd', 's4'}


{'even', 'odd'}

In [5]:
def reduction(acceptor: Acceptor):
    def split(state_class, states, symbol):
        return {state_class_1 for state_class_1 in state_class
                if acceptor.transition_function[state_class_1][symbol] in states}, \
               {state_class_2 for state_class_2 in state_class
                if acceptor.transition_function[state_class_2][symbol] not in states}

    classes: List[Set[str]] = [acceptor.accepting_states, acceptor.states - acceptor.accepting_states]
    states_symbol_pairs_queue = queue.Queue()
    for symbol in acceptor.alphabet:
        states_symbol_pairs_queue.put((acceptor.accepting_states, symbol))
        states_symbol_pairs_queue.put((acceptor.states - acceptor.accepting_states, symbol))
    while not states_symbol_pairs_queue.empty():
        (states, symbol) = states_symbol_pairs_queue.get()
        for state_class in classes:
            state_class_1, state_class_2 = split(state_class, states, symbol)
            if state_class_1 and state_class_2:
                classes.remove(state_class)
                classes.append(state_class_1)
                classes.append(state_class_2)
                for token in acceptor.alphabet:
                    states_symbol_pairs_queue.put((state_class_1, token))
                    states_symbol_pairs_queue.put((state_class_2, token))
    new_states = {state_class.pop() for state_class in classes}
    return Acceptor(acceptor.alphabet, new_states, acceptor.initial_state, acceptor.accepting_states, acceptor.transition_function)
  
alphabet = {"1", "0"}
states = {"even", "odd", "odd2" "s3", "s4"}
initial_state = "even"
accepting_states = {"even"}
transition_function = {
    "even": {
        "0": "even",
        "1": "odd"
    },
    "odd": {
        "0": "odd",
        "1": "even"
    },
    "odd": {
        "0": "odd",
        "1": "even"
    }
}